In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON
import datetime

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
song_path = Path(params["song_path"])
import scipy.io.wavfile
fs, data = scipy.io.wavfile.read(song_path)
data = np.squeeze(data)
if data.ndim != 1:
    raise Exception("Expected wav files of length 1")
duration = data.size /fs
ds = xr.Dataset()
ds["song"] = xr.DataArray(data, dims="t")
ds["t"] = np.arange(data.size)/fs
ds["t"].attrs["fs"] = fs
display(fs)
display(duration)
display(ds)
new_fs = int(np.round(fs))

In [ ]:


if not np.abs(fs - new_fs) > 10**(-5):
    display("Outputing wav files with integer fs, interpolating...")
    ds["resampled_song"] = ds["song"].interp(dict(t=np.arange(int(ds["t"].max().item()*new_fs))/new_fs)).rename(t="new_t")
    ds["new_t"].attrs["fs"] = new_fs
else:
    ds["resampled_song"] = ds["filtered_song"].rename(t="new_t")
    ds["new_t"].attrs["fs"] = new_fs
ds

In [ ]:
import scipy.io.wavfile
M = np.abs(ds["resampled_song"]).max().item()
scipy.io.wavfile.write("../"+params["out_name"], ds["new_t"].attrs["fs"], (ds["resampled_song"].to_numpy()* (32767/M)).astype(np.int16))

In [ ]:
metadata = {k:v for k,v in params.items() if not k in ["song_path", "out_name"]}
metadata["duration"] = duration
metadata["original_fs"] = fs
metadata["fs"] = fs
metadata["value_rescaling"] = 32767/M
display(RenderJSON(metadata))
metadata["date"] = datetime.datetime.strptime(metadata["date"], '%Y-%m-%d %H:%M:%S.%f')
with Path("../metadata.yaml").open("w") as f:
    yaml.safe_dump(metadata, f)